In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from urllib.parse import urlencode
import datetime

In [3]:
# импортируем данные по ссылкам из Яндекс Диска

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'

In [4]:
public_key = 'https://disk.yandex.ru/d/Ta-weYfGM5CHjg'
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

Выгружены данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.


Описание данных 

- date – дата совершения события
- event - событие

app_install – установка приложения
app_start – открыть приложения
registration – зарегистрироваться 
search – перейти на страницу поиска товаров (каталог)
open_item – открыть товар
choose_item – отправить товар в корзину
tap_basket – перейти в корзину
purchase – подтверждение покупки

- gender – пол пользователя
- os_name – платформа пользователя
- city – город пользователя
- device_id – идентификатор устройства пользователя
- urm_source – канал, с которого пришел пользователь

yandex-direct – Яндекс директ
google_ads – реклама в Google
vk_ads – реклама в ВК
instagram_ads – реклама в instagram
facebook_ads – реклама в facebook
referral – акция «приведи друга»
Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы  

- purchase_sum – стоимость покупки (при совершении события ‘purchase’)

Обратите внимание на следующее:
- В выгрузке только уникальные действия пользователей за каждый день 
- Можно миновать стадию установки приложения, если оно было установлено ранее
- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку.

In [5]:
data = pd.read_csv(download_url)

In [6]:
data.shape

(2747968, 8)

In [7]:
data['date'] = pd.to_datetime(data['date'])

In [8]:
data.dtypes

date            datetime64[ns]
event                   object
purchase_sum           float64
os_name                 object
device_id                int64
gender                  object
city                    object
utm_source              object
dtype: object

In [9]:
data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [10]:
# MAU февраля:

data \
    .query("'2020-02-01' <= date <= '2020-02-29'") \
    .device_id \
    .nunique()

75032

In [11]:
# Количество установок в январе:

data \
    .query("'2020-01-01' <= date <= '2020-01-31' and event == 'app_install'") \
    .shape[0]

80297

In [12]:
# Присвойте пользователям когорты по дню установки приложения.
# Посчитайте для них  конверсию из установки в покупку в течение 7 дней.
# Для какой когорты конверсия была наибольшей? Ответ впишите в формате: дд.мм.гггг

# Примечание: считаем пользователя сконвертировавшимся,
# если с момента установки до совершения первой покупки прошло не более 7 дней.

install_events = \
    data \
        .query("event == 'app_install'")

In [13]:
install_events.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
2515061,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads
2515062,2020-01-01,app_install,NaN,ios,1311583,male,Moscow,-
2515063,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-
2515064,2020-01-01,app_install,NaN,android,5645652,male,Moscow,-
2515065,2020-01-01,app_install,NaN,ios,8798567,male,Saint-Petersburg,referal


In [14]:
# для каждого клиента находим дату его первой покупки

first_buy = \
data \
    .query("event == 'purchase' and device_id in @install_events.device_id") \
    .groupby('device_id') \
    .agg({'date': 'min'}) \
    .reset_index() \
    .rename(columns={'date': 'first_buy'})

In [15]:
first_buy

,device_id,first_buy
0,4046,2020-01-04
1,4120,2020-01-07
2,4125,2020-01-07
3,4202,2020-02-27
4,4234,2020-02-19
...,...,...
57453,35315632,2020-03-31
57454,35321120,2020-03-25
57455,35342310,2020-03-27
57456,35367797,2020-03-29


In [16]:
users = install_events[['date', 'device_id']].merge(first_buy, how='left', on='device_id')

In [17]:
users = users \
    .rename(columns={'date': 'install_date'})

In [18]:
users = users[['device_id', 'install_date', 'first_buy']]

In [19]:
users.head()

,device_id,install_date,first_buy
0,4921563,2020-01-01,2020-01-10
1,1311583,2020-01-01,NaT
2,23006325,2020-01-01,2020-01-01
3,5645652,2020-01-01,NaT
4,8798567,2020-01-01,NaT


In [20]:
users['days'] = users['first_buy'] - users['install_date']

In [21]:
users.head()

,device_id,install_date,first_buy,days
0,4921563,2020-01-01,2020-01-10,9 days
1,1311583,2020-01-01,NaT,NaT
2,23006325,2020-01-01,2020-01-01,0 days
3,5645652,2020-01-01,NaT,NaT
4,8798567,2020-01-01,NaT,NaT


In [22]:
df_1 = \
users \
    .query("days <= datetime.timedelta(days=7)") \
    .groupby('install_date') \
    .agg({'device_id': 'count'}) \
    .rename(columns={'device_id': 'less_than_7_days'}) \
    .reset_index()

In [23]:
df_2 = \
users \
    .groupby('install_date') \
    .agg({'device_id': 'count'}) \
    .rename(columns={'device_id': 'total_amount'}) \
    .reset_index()

In [24]:
df = df_1.merge(df_2, on='install_date')

In [25]:
df['CR'] = df['less_than_7_days'] / df['total_amount']

In [26]:
df['CR'] = df['CR'] * 100

In [27]:
df

,install_date,less_than_7_days,total_amount,CR
0,2020-01-01,1408,3579,39.340598
1,2020-01-02,1186,3144,37.722646
2,2020-01-03,834,2402,34.721066
3,2020-01-04,639,1831,34.898962
4,2020-01-05,587,1671,35.128665
...,...,...,...,...
86,2020-03-27,277,1199,23.102585
87,2020-03-28,209,1091,19.156737
88,2020-03-29,223,1117,19.964190
89,2020-03-30,204,994,20.523139


In [28]:
df[df.CR == df.CR.max()]

,install_date,less_than_7_days,total_amount,CR
0,2020-01-01,1408,3579,39.340598


In [29]:
# С какого платного маркетингового канала пришло больше всего новых пользователей?

data \
    .query("event == 'app_install'") \
    .utm_source \
    .value_counts()

-                32460
yandex-direct    29368
google_ads       26286
vk_ads           23189
instagram_ads    20096
facebook_ads     13916
referal           9282
Name: utm_source, dtype: int64

In [30]:
# Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов.
# Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. 
# На каком шаге отваливается больше всего зарегистрированных пользователей?

# В ответ впишите один из шагов, конверсия из которого (в следующий шаг) самая низкая, в таком формате:
# «Поиск», «Добавление товара», «Переход в корзину», «Регистрация».

# Подсказка:
# - Для того, чтобы выделить группу тех, кому нужно и не нужно регистрироваться, добавьте колонку с датой регистрации.
# Если дата регистрации < даты совершения события, то пользователь уже зарегистрирован.

In [48]:
data.query("event == 'register'")[['device_id', 'date']]

,device_id,date
2669658,294193,2020-01-01
2669659,22917617,2020-01-01
2669660,15248490,2020-01-01
2669661,252062,2020-01-01
2669662,2251583,2020-01-01
...,...,...
2747963,2984778,2020-03-31
2747964,27301864,2020-03-31
2747965,1294285,2020-03-31
2747966,3010574,2020-03-31


In [55]:
data = data \
    .merge(data.query("event == 'register'")[['device_id', 'date']],
              how='left',
              on='device_id') \
    .rename(columns={'date_y': 'registration_date'})

KeyError: "['date'] not in index"

In [56]:
data.head()

,date_x,event,purchase_sum,os_name,device_id,gender,city,utm_source,registration_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,NaT


In [ ]:
# смотрим воронку для зарегистрированных пользователей

In [61]:
data.query("event == 'search' and date_x > registration_date").shape[0]

417101

In [62]:
data.query("event == 'choose_item' and date_x > registration_date").shape[0]

314473

In [63]:
# CR из поиска в добавление товара в корзину
314473 / 417101 * 100

75.39492832671225

In [64]:
data.query("event == 'tap_basket' and date_x > registration_date").shape[0]

217256

In [65]:
# CR из добавления товара в корзину в переход в корзину
217256 / 314473 * 100

69.08574027022988

In [68]:
data.query("event == 'purchase' and date_x > registration_date").shape[0]

73630

In [70]:
# CR из перехода в корзину в оплату
73630 / 217256 * 100

33.890893692234044

In [74]:
# Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

# В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

# Подсказка: 
# - CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение 

inorganic_users = data.query("utm_source != '-'")

In [72]:
inorganic_users.shape

(2137510, 9)

In [73]:
inorganic_users.head()

,date_x,event,purchase_sum,os_name,device_id,gender,city,utm_source,registration_date
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,NaT
6,2020-01-01,app_start,NaN,android,8383386,female,Saint-Petersburg,vk_ads,2020-01-23


In [84]:
first_start = \
    inorganic_users \
        .query("event == 'app_start'") \
        .groupby('device_id') \
        .agg({'date_x': 'min'}) \
        .reset_index() \
        .rename(columns={'date_x': 'first_start_date'})

In [86]:
first_start.head()

,device_id,first_start_date
0,4013,2020-01-15
1,4014,2020-01-02
2,4016,2020-01-04
3,4046,2020-01-08
4,4060,2020-01-03


In [89]:
first_start.shape

(158283, 2)

In [90]:
inorg_users = inorganic_users.query("event == 'purchase'")[['date_x', 'event', 'device_id', 'utm_source']]

In [93]:
inorg_users = inorg_users \
    .groupby('device_id') \
    .agg({'date_x': 'min'}) \
    .reset_index()

In [95]:
start_and_buy_users = first_start.merge(inorg_users, how='left', on='device_id')

In [97]:
start_and_buy_users.shape

(158283, 3)

In [96]:
start_and_buy_users.head()

,device_id,first_start_date,date_x
0,4013,2020-01-15,NaT
1,4014,2020-01-02,2020-02-01
2,4016,2020-01-04,NaT
3,4046,2020-01-08,2020-01-11
4,4060,2020-01-03,2020-03-07


In [103]:
inorganic_users[['device_id', 'utm_source']]

,device_id,utm_source
1,833621,vk_ads
2,1579237,referal
3,1737182,facebook_ads
4,4029024,facebook_ads
6,8383386,vk_ads
...,...,...
2747961,4294936,yandex-direct
2747962,9613546,google_ads
2747963,2984778,facebook_ads
2747965,1294285,google_ads


In [106]:
final_df = start_and_buy_users.merge(inorganic_users[['device_id', 'utm_source']].drop_duplicates(subset='device_id'),
                          how='left',
                          on='device_id')

In [107]:
final_df

,device_id,first_start_date,date_x,utm_source
0,4013,2020-01-15,NaT,vk_ads
1,4014,2020-01-02,2020-02-01,referal
2,4016,2020-01-04,NaT,yandex-direct
3,4046,2020-01-08,2020-01-11,facebook_ads
4,4060,2020-01-03,2020-03-07,yandex-direct
...,...,...,...,...
158278,35379281,2020-03-29,NaT,vk_ads
158279,35380796,2020-03-31,NaT,referal
158280,35381595,2020-03-31,NaT,facebook_ads
158281,35388218,2020-03-31,NaT,yandex-direct


In [111]:
final_df['is_conversed'] = np.where(pd.isnull(final_df.date_x), 0, 1)

In [112]:
final_df

,device_id,first_start_date,date_x,utm_source,is_conversed
0,4013,2020-01-15,NaT,vk_ads,0
1,4014,2020-01-02,2020-02-01,referal,1
2,4016,2020-01-04,NaT,yandex-direct,0
3,4046,2020-01-08,2020-01-11,facebook_ads,1
4,4060,2020-01-03,2020-03-07,yandex-direct,1
...,...,...,...,...,...
158278,35379281,2020-03-29,NaT,vk_ads,0
158279,35380796,2020-03-31,NaT,referal,0
158280,35381595,2020-03-31,NaT,facebook_ads,0
158281,35388218,2020-03-31,NaT,yandex-direct,0


In [114]:
final_df \
    .groupby('utm_source') \
    .agg({'is_conversed': 'mean'}) \
    .sort_values('is_conversed')

,is_conversed
utm_source,
yandex-direct,0.343438
google_ads,0.343914
facebook_ads,0.358706
instagram_ads,0.368752
vk_ads,0.388420
referal,0.407112


In [120]:
# Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

In [125]:
data.head()

,date_x,event,purchase_sum,os_name,device_id,gender,city,utm_source,registration_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,NaT


In [126]:
first_purchase_date = data.query("event == 'purchase'") \
    .groupby('device_id') \
    .agg({'date_x': 'min'}) \
    .reset_index() \
    .rename(columns={'date_x': 'first_purchase_date'})

In [127]:
first_purchase_date.head()

,device_id,first_purchase_date
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4120,2020-01-07
4,4125,2020-01-07


In [129]:
purchase_df = first_purchase_date.merge(data, how='left', on='device_id')

In [132]:
purchase_df = purchase_df.query("event == 'purchase' and first_purchase_date == date_x")

In [137]:
purchase_df \
    .groupby('utm_source') \
    .agg({'purchase_sum': 'median'}) \
    .sort_values('purchase_sum')

,purchase_sum
utm_source,
facebook_ads,389.0
google_ads,390.5
yandex-direct,392.5
vk_ads,393.0
instagram_ads,393.5
referal,395.5
-,398.5


In [141]:
# Данные по затратам на рекламу:

# Яндекс – 10 491 707 руб.
# Гугл – 10 534 878 руб.
# Фейсбук – 8 590 498 руб.
# Инстаграм – 8 561626 руб.
# ВК – 9 553 531 руб.

# Какой платный канал привлечения имеет самый высокий ROMI?

In [139]:
data \
    .groupby('utm_source') \
    .agg({'purchase_sum': 'sum'})

,purchase_sum
utm_source,
-,21449749.5
facebook_ads,12249901.0
google_ads,12868276.0
instagram_ads,14546969.0
referal,8837044.5
vk_ads,16389652.5
yandex-direct,13915368.0


In [143]:
# ROMI Фейсбук

(12249901 - 8590498) / 8590498

0.4259826380263403

In [144]:
# ROMI Гугл

(12868276 - 10534878) / 10534878

0.22149264566708793

In [145]:
# ROMI Инстаграм

(14546969 - 8561626) / 8561626

0.6990895187432854

In [146]:
# ROMI ВКонтакте

(16389652 - 9553531) / 9553531

0.7155596187420128

In [147]:
# ROMI Яндекс

(13915368 - 10491707) / 10491707

0.3263206835646478